# Code for Q2

In [33]:
Positions = [0.0, 0.5, 1.2, 2.0]
Temp      = [150.0, 165.0, 172.0, 185.0]

# Build only a 4x4 table: Col0 = x, Col1 = T, Col2 = first diff, Col3 = second diff
n = len(Positions)
table = [[0.0]*5 for _ in range(n)]

# Col 0: positions
for i, x in enumerate(Positions):
    table[i][0] = x

# Col 1: temperatures
for i, t in enumerate(Temp):
    table[i][1] = t

# Col 2: first divided differences (start at row 1)
for i in range(1, n):
    table[i][2] = (table[i][1] - table[i-1][1]) / (table[i][0] - table[i-1][0])

# Col 3: second divided differences (start at row 2)
# f[x_{i-2}, x_{i-1}, x_i] = (f[x_{i-1}, x_i] - f[x_{i-2}, x_{i-1}]) / (x_i - x_{i-2})
for i in range(2, n):
    table[i][3] = (table[i][2] - table[i-1][2]) / (table[i][0] - table[i-2][0])

#Col 4:
for i in range(3,n):
    table[i][4] = (table[i][3]-table[i-1][3])/table[i][0]-table[i-3][0]
# Pretty square print (4x4)
print("Square divided-difference table (unused entries shown as 0.0):")
header = ["x", "T", "1st Δ", "2nd Δ", "3rd Δ"]
print(" | ".join(f"{h:^10}" for h in header))
print("-"*62)
x0 = table[0][0]
x1 = table[1][0]
x2 = table[2][0]
x3 = table[3][0]
fx0 = table[0][1]
fx0x1 = table[1][2]
fx0x1x2 = table[2][3]
fx0x1x2x3 = table[3][4]

for row in table:
    print(" | ".join(f"{val:10.6f}" for val in row))
print(fx0," ",fx0x1," ",fx0x1x2," ",fx0x1x2x3)
#Our interpolating Polynomial has been derived as:
def polynomial(x):
    return fx0 + (x-x0)*fx0x1 + (x-x0)*(x-x1)*fx0x1x2 + (x-x0)*(x-x1)*(x-x2)*fx0x1x2x3
#Value required:
print("Value of P(x) at x = 1 is:=", polynomial(1))

Square divided-difference table (unused entries shown as 0.0):
    x      |     T      |   1st Δ    |   2nd Δ    |   3rd Δ   
--------------------------------------------------------------
  0.000000 | 150.000000 |   0.000000 |   0.000000 |   0.000000
  0.500000 | 165.000000 |  30.000000 |   0.000000 |   0.000000
  1.200000 | 172.000000 |  10.000000 | -16.666667 |   0.000000
  2.000000 | 185.000000 |  16.250000 |   4.166667 |  10.416667
150.0   30.0   -16.666666666666668   10.416666666666668
Value of P(x) at x = 1 is:= 170.625


In [ ]:
#Now, by solving system of linear equations to obtain the interpolating polynomial
# 4x4 zero matrix

# Build 4x4 zero matrix then fill
A = [[0.0]*4 for _ in range(4)]
for i, x in enumerate(Positions):
    A[i][0] = 1.0      # d coefficient
    A[i][1] = x        # c coefficient
    A[i][2] = x**2     # b coefficient
    A[i][3] = x**3     # a coefficient

b = Temp[:]  # RHS

print("Original augmented matrix [A|b] with unknowns [d c b a]^T:")
for row, bi in zip(A, b):
    print("  " + "  ".join(f"{v:10.6f}" for v in row) + " | " + f"{bi:10.6f}")

def row_echelon(A, b):
    M = [row[:] for row in A]
    r = b[:]
    m = len(M)
    n = len(M[0])
    pivot_row = 0
    for col in range(n):
        # Find pivot
        piv = max(range(pivot_row, m), key=lambda i: abs(M[i][col]))
        if abs(M[piv][col]) < 1e-14:
            continue
        # Swap if needed
        if piv != pivot_row:
            M[pivot_row], M[piv] = M[piv], M[pivot_row]
            r[pivot_row], r[piv] = r[piv], r[pivot_row]
        # Normalize pivot row (optional but nicer)
        piv_val = M[pivot_row][col]
        for j in range(col, n):
            M[pivot_row][j] /= piv_val
        r[pivot_row] /= piv_val
        # Eliminate below
        for i in range(pivot_row+1, m):
            factor = M[i][col]
            if factor != 0.0:
                for j in range(col, n):
                    M[i][j] -= factor * M[pivot_row][j]
                #Elimination Step:
                r[i] -= factor * r[pivot_row]
        pivot_row += 1
        if pivot_row == m:
            break
    return M, r

U, rb = row_echelon(A, b)

print("\nRow echelon form [U|b']:")
for row, bi in zip(U, rb):
    print("  " + "  ".join(f"{v:10.6f}" for v in row) + " | " + f"{bi:10.6f}")

#back substitution to get coefficients [d, c, b, a]
def back_sub(U, rb):
    n = len(U)
    x = [0.0]*n
    for i in range(n-1, -1, -1):
        s = rb[i] - sum(U[i][j]*x[j] for j in range(i+1, n))
        # Find pivot column
        pivot_col = next((k for k,v in enumerate(U[i]) if abs(v) > 1e-12), None)
        if pivot_col is None:
            continue
        x[pivot_col] = s / U[i][pivot_col]
    return x

coeffs = back_sub(U, rb)  # [d, c, b, a]
d, c, b2, a = coeffs
print(f"\nCoefficients (order [d, c, b, a]): d={d}, c={c}, b={b2}, a={a}")

# Build readable interpolating polynomial:
poly_str = f"T(x) = {a:.10f} x^3 + {b2:.10f} x^2 + {c:.10f} x + {d:.10f}"
print("\nInterpolating polynomial:")
print(poly_str)

# Quick verification
x_test = 0.0
T_test = a*x_test**3 + b2*x_test**2 + c*x_test + d
print(f"T(0.0) = {T_test:.10f}")
x_test = 0.5
T_test = a*x_test**3 + b2*x_test**2 + c*x_test + d
print(f"T(0.5) = {T_test:.10f}")
x_test = 1.2
T_test = a*x_test**3 + b2*x_test**2 + c*x_test + d
print(f"T(1.2) = {T_test:.10f}")
x_test = 2.0
T_test = a*x_test**3 + b2*x_test**2 + c*x_test + d
print(f"T(2.0) = {T_test:.10f}")
x_test = 1.0
T_test = a*x_test**3 + b2*x_test**2 + c*x_test + d
print(f"T(1.0) = {T_test:.10f}")
#This shows that our interpolating Polynomial found via Newton's D.D yields same value!
#Our interpolating Polynomial is always unique! Proof for Uniqueness is written in the report
# --- Theoretical note (why this works) ---
# The 4x4 matrix A is a Vandermonde-like matrix with rows [1, x_i, x_i^2, x_i^3].
# Because all x_i are distinct, the columns are linearly independent -> A has full rank (rank = 4),
# so the linear system A * [d c b a]^T = b has a unique solution.
# If the matrix were NOT full rank (e.g., repeated x values), either:
#   (a) The system would be inconsistent (no polynomial of degree <=3 fits all 4 data),
#   (b) Or there would be infinitely many solutions (data determined by a lower-degree polynomial),
# depending on whether the right-hand side lies in the column space of A.
# Full rank guarantees uniqueness of the interpolating cubic.

Original augmented matrix [A|b] with unknowns [d c b a]^T:
    1.000000    0.000000    0.000000    0.000000 | 150.000000
    1.000000    0.500000    0.250000    0.125000 | 165.000000
    1.000000    1.200000    1.440000    1.728000 | 172.000000
    1.000000    2.000000    4.000000    8.000000 | 185.000000

Row echelon form [U|b']:
    1.000000    0.000000    0.000000    0.000000 | 150.000000
    0.000000    1.000000    2.000000    4.000000 |  17.500000
    0.000000    0.000000    1.000000    3.200000 |  -1.041667
    0.000000    0.000000    0.000000    1.000000 |  10.416667

Coefficients (order [d, c, b, a]): d=150.0, c=44.58333333333331, b=-34.37499999999997, a=10.416666666666659

Interpolating polynomial:
T(x) = 10.4166666667 x^3 + -34.3750000000 x^2 + 44.5833333333 x + 150.0000000000
T(0.0) = 150.0000000000
T(0.5) = 165.0000000000
T(1.2) = 172.0000000000
T(2.0) = 185.0000000000
T(1.0) = 170.6250000000
